# Statistiques sur les prénoms aux USA


Données accessibles sur le site de la Sécurité Sociale US.

Prénom, genre et effectif des enfants nés aux USA depuis 1880.

Seuls les prénoms avec au moins 5 naissances dans une année sont présents.

In [ ]:
from IPython.display import HTML
HTML('<iframe src="https://www.ssa.gov/oact/babynames/limits.html" width=800 height=400></iframe>')

In [ ]:
# imports
import pandas as pnd
import numpy as np
import os
import matplotlib.pyplot as plt

# commande magique pour l'affichage des graphiques
%matplotlib inline

# options d'affichage
from pandas import set_option
set_option("display.max_rows", 16)
plt.style.use('seaborn-notebook')

## 1. Chargement des données d'une seule année

In [ ]:
# chargement des données d'une année dans un DataFrame
path = "C:/Test/mydata/names"
filename = "yob2015.txt"
df2015 = pnd.read_csv(os.path.join(path, filename),
                           header=None,
                           names=['prénom', 'genre', 'naissances'])
df2015.head()

In [ ]:
# aide en ligne
pnd.read_csv?

In [ ]:
# dimensions du DataFrame
df2015.shape

In [ ]:
# naissances par genre en 2015
df2015.groupby("genre").sum()

In [ ]:
# les 10 prénoms les plus donnés en 2015
df2015.sort_values(by='naissances', ascending=False).head(10)

## 2. Chargement de toutes les données

In [ ]:
# chargement de toute les données dans un DataFrame
df = pnd.DataFrame()
for root, dirs, files in os.walk(path):
    for filename in files:
        annee = int(filename[3:7]) # yobAAAA.txt
        csv = pnd.read_csv(os.path.join(root, filename),
                           header=None,
                           names=['prénom', 'genre', 'naissances'])
        csv['année'] = annee
        df = df.append(csv, ignore_index=True)
df = df[['année', 'prénom', 'genre', 'naissances']]
# tête du DataFrame
df.head()

In [ ]:
# queue du DataFrame
df.tail()

In [ ]:
# taille du DataFrame
df.shape

In [ ]:
# les 10 prénoms les plus donnés dans une année
df.sort_values(by='naissances', ascending=False).head(10)

In [ ]:
# nombre de naissances par genre
df.pivot_table(index='genre', values=['naissances'], aggfunc=np.sum)

In [ ]:
# les 10 prénoms + genre les plus donnés
data = df.pivot_table(index=['prénom','genre'], values=['naissances'], aggfunc=np.sum)
data.sort_values(by='naissances', ascending=False, inplace=True)
data.reset_index(inplace=True)
data.head(10)

In [ ]:
# recherche d'une chaine
def cherche(chaine):
    global data
    return data[data['prénom'].str.startswith(chaine)].head(10)

In [ ]:
cherche("Jo") # Nath Fran Em

In [ ]:
# naissances par année et par genre
total = df.pivot_table(index='année', columns=['genre'], values='naissances', aggfunc=np.sum)
total

In [ ]:
# graphique
total.plot(title="Total des naissances par année et par genre", color=['r','b']);

In [ ]:
# évolution d'un prénom
def stats_prenom(prenom, genre):
    global df
    data = df[(df['prénom']==prenom)&(df['genre']==genre)]
    pivot = data.pivot_table(index='année', values='naissances')
    pivot.plot(title="Total des naissances de %s (%s) par année" % (prenom, genre));

In [ ]:
stats_prenom('Sarah', 'F')

In [ ]:
stats_prenom('Barack', 'M')

## 3. Analyse de tendances

In [ ]:
# regroupement par année et genre
for nom, groupe in df.groupby(['année', 'genre']):
    print(nom)
    print(groupe)
    break

In [ ]:
def ajout_proportion(groupe):
    # Integer division floors
    naissances = groupe["naissances"].astype(float)
    groupe['proportion'] = naissances / naissances.sum()
    return groupe

df = df.groupby(['année', 'genre']).apply(ajout_proportion)
df.head()

In [ ]:
# on vérifie que la somme des proportions par année et par genre vaut 1
df.groupby(['année', 'genre'])["proportion"].sum()==1

In [ ]:
# vérification : la somme des proportions par année et par genre vaut presque 1
np.allclose(df.groupby(['année', 'genre'])["proportion"].sum(), 1)

In [ ]:
# sélection du top 1000 des prénoms + genre par année
def top1000(groupe):
    return groupe.sort_values(by='naissances', ascending=False)[:1000]

df1000 = df.groupby(['année', 'genre']).apply(top1000)
df1000.head()

In [ ]:
# taille du DataFrame
df1000.shape

In [ ]:
# évolution des prénoms par année
data = df1000.pivot_table(index='année', columns='prénom', values='naissances', aggfunc=sum)
data = data[['John', 'Harry', 'Mary', 'Marilyn']]
data.head()

In [ ]:
# graphique
data.plot(subplots=True, figsize=(12, 10), grid=False, title="Nombre de naissances par an");

## 4. Mesure de l'évolution de la diversité des prénoms

In [ ]:
# nombre de prénoms différents par année et genre
data = df.pivot_table(index='année', columns='genre', values='prénom', aggfunc=lambda x: len(x))
data.head()

In [ ]:
data.tail()

In [ ]:
# graphique
data.plot(title="Evolution du nombre de prénoms distincts par genre",
         color=['r', 'b']);

In [ ]:
# proportion des naissances qui sont dans le top 1000 par année et par genre
data = df1000.pivot_table(index='année', columns='genre', values='proportion', aggfunc=sum)
data.head()

In [ ]:
data.tail()

In [ ]:
# graphique
data.plot(title='Proportion des naissances dans le top 1000 par année et genre',
          xticks=range(1880, 2020, 10),
          yticks=np.linspace(0, 1.2, 13),
          figsize=(12, 6),
          color=['r','b']);

In [ ]:
# sous-ensembles par genre
df1 = df1000[df1000["genre"]=="M"]
df1.head()

In [ ]:
# année 2015
df1_2015 = df1[df1["année"]==2015]
df1_2015

In [ ]:
# cumul de de la proportion des naissances pour 2015 + M
cumul1_2015 = df1_2015.sort_values(by='proportion', ascending=False)["proportion"].cumsum()
cumul1_2015

In [ ]:
# recherche du rang correspondant à un cumul de 0.5 (50%)
cumul1_2015.searchsorted(0.5)[0]

In [ ]:
df1[df1["année"]==1880].sort_values(by='proportion', ascending=False)["proportion"].cumsum().searchsorted(0.5)[0]

In [ ]:
# fonction de calcul du rang correspondant à un cumul de q
def quantile_compte(groupe, q=0.5):
    g = groupe.sort_values(by='proportion', ascending=False)
    return g["proportion"].cumsum().searchsorted(q)[0] + 1

In [ ]:
# calcul de la diversité
diversite = df1000.groupby(['année', 'genre']).apply(quantile_compte)
diversite = diversite.unstack('genre')
diversite.head()

In [ ]:
# graphique
diversite.plot(title="Nombre de prénoms populaires dans le top 50%",
          color=['r','b']);

## 5. Evolution de la dernière lettre des prénoms

In [ ]:
# ajout d'une colonne avec la lettre terminale
df["terminale"] = df["prénom"].apply(lambda x: x[-1])
df.head()

In [ ]:
# tableau du nombre de prénoms par année x lettre terminale
data = df.pivot_table(index='année', columns='terminale', values='prénom', aggfunc=len)
data.head()

In [ ]:
data.tail()

In [ ]:
# les 7 lettres terminales les plus utilisées en 2015
lettres = data.loc[2015].sort_values(ascending=False).head(7).index.values
lettres

In [ ]:
# graphique
data[lettres].plot(title="Evolution du nombre de prénoms distincts en fonction de leur lettre terminale");

In [ ]:
# tableau letres terminales x (genres, années)
data = df.pivot_table(index='terminale', columns=['genre', 'année'], values='naissances', aggfunc=sum)
data.head()

In [ ]:
# extrait pour les années 1910, 1960 et 2010
subdata = data.reindex(columns=[1910, 1960, 2010], level='année')
subdata

In [ ]:
# total par année
subdata.sum()

In [ ]:
# proportion du nombre de prénoms par lettre terminale
proportion_lettre = subdata / subdata.sum().astype(float)
proportion_lettre

In [ ]:
# graphique
fig, axes = plt.subplots(2, 1, figsize=(10, 8))
proportion_lettre['M'].plot(kind='bar', rot=0, ax=axes[0], title='Masculin')
proportion_lettre['F'].plot(kind='bar', rot=0, ax=axes[1], title='Féminin', legend=False);

In [ ]:
# proportion du nombre de prénoms par lettre terminale
proportion_lettre = data / data.sum().astype(float)
proportion_lettre

In [ ]:
# proportion pour les lettres d, n et y (M)
dny = proportion_lettre.ix[['d', 'n', 'y'], 'M'].T
dny

In [ ]:
# graphique
dny.plot(title="Proportion des hommes avec des prénoms se terminant en d, n ou y");

In [ ]:
# proportion pour les lettres a, e et n (F)
aen = proportion_lettre.ix[['a', 'e', 'n'], 'F'].T
aen

In [ ]:
# graphique
aen.plot(title="Proportion des femmes avec des prénoms se terminant en a, e ou n");

## 6. Prénoms dont le genre a évolué

In [ ]:
# ratio par prénom et par an du genre F
data = df1000.pivot_table(index=['prénom','année'], columns=['genre'], values='naissances', aggfunc=np.sum)
data.dropna(inplace=True)
data["ratio"] = data["F"] / (data["F"]+data["M"])
data.reset_index(inplace=True)
data

In [ ]:
# variance du ratio par prénom
data = data.pivot_table(index='prénom', values=['ratio'], aggfunc=np.var)
data.dropna(inplace=True)
data.sort_values(by="ratio", ascending=False, inplace=True)
data.head()

In [ ]:
# sélection des données pour le prénom leslie
data = df[df["prénom"]=="Leslie"]
data

In [ ]:
# tableau du ratio de naissances de Leslie par année et par genre
table = data.pivot_table(index='année', columns='genre', values='naissances', aggfunc='sum')
table = table.div(table.sum(1), axis=0)
table

In [ ]:
# graphique
table.plot(style={'M': 'b', 'F': 'r'}, title="Evolution de la proportion du prénom Leslie par genre");

In [ ]:
# fonction de calcul de l'évolution d'un prénom par genre
def evolution(prenom):
    data = df[df["prénom"]==prenom]
    table = data.pivot_table(index='année', columns='genre', values='naissances', aggfunc='sum')
    table = table.div(table.sum(1), axis=0)
    table.plot(style={'M': 'b', 'F': 'r'}, title="Evolution de la proportion du prénom %s par genre" % prenom);

In [ ]:
evolution("Sidney")

## 7. Comparaison du nombre de naissances entre 2 années sucessives

In [ ]:
# années 2014 et 2015
var = df[df["année"].isin([2014,2015])].pivot_table(index="prénom",columns="année",values="naissances")
var.fillna(0, inplace=True)
var /=1000
var.head()

In [ ]:
# graphique
var.plot(kind='scatter', x=2014, y=2015);

In [ ]:
# régression linéaire
from scipy.stats import linregress
slope, intercept, r, p, stderr = linregress(var[2014], var[2015])
slope, intercept

In [ ]:
# calcul de l'erreur
var["erreur"] = var[2015] - var[2014] * slope - intercept

In [ ]:
# calcul du min et du max
var["erreur"].min(), var["erreur"].max()

In [ ]:
# qualification de l'écart
var["écart"] = pnd.cut(var["erreur"], [-10, -0.5, 0.5, 10], labels=["négatif", "neutre", "positif"])
var.head()

In [ ]:
# régression linéaire sur les écarts positifs
var_hausse = var[var["écart"]=="positif"]
slope_h, intercept_h, r_h, p_h, stderr_h = linregress(var_hausse[2014], var_hausse[2015])
slope_h, intercept_h

In [ ]:
# régression linéaire sur les écarts négatifs
var_baisse = var[var["écart"]=="négatif"]
slope_b, intercept_b, r_b, p_b, stderr_b = linregress(var_baisse[2014], var_baisse[2015])
slope_b, intercept_b

In [ ]:
# graphique
plt.scatter(x=var[2014], y=var[2015]);
x = np.linspace(0,12,2)
b = slope_b * x + intercept_b
n = slope * x + intercept
h = slope_h * x + intercept_h
plt.plot(x, b, 'r');
plt.plot(x, n, 'y');
plt.plot(x, h, 'g');

In [ ]:
# graphique
plt.scatter(x=var[2014], y=var[2015]);
x = np.linspace(0,12,2)
b = slope_b * x + intercept_b
n = slope * x + intercept
h = slope_h * x + intercept_h
plt.plot(x, b, 'r');
plt.plot(x, n, 'y');
plt.plot(x, h, 'g');

In [ ]:
# graphique
plt.scatter(x=var[2014], y=var[2015]);
x = np.linspace(0,12,2)
b = slope_b * x + intercept_b
n = slope * x + intercept
h = slope_h * x + intercept_h
plt.plot(x, b, 'r');
plt.plot(x, n, 'y');
plt.plot(x, h, 'g');

# Données sur les prénoms français

Données accessibles à cette adresse : https://www.insee.fr/fr/statistiques/2540004

## Chargement des données

In [ ]:
# chargement des données
df = pnd.read_csv("nat2015.txt", sep='\t', encoding='latin-1')
df

In [ ]:
df = pnd.read_csv("nat2015.txt", sep='\t', encoding='latin-1', header=0,
                  names = ['genre', 'prénom', 'année', 'naissances'],
                  converters = {'genre': (lambda x: 'M' if x=='1' else 'F'),\
                               'prénom': (lambda x: '-'.join(map(lambda y: y.capitalize(), x.split('-')))),\
                               'année': (lambda x: int(x) if x != 'XXXX' else x),\
                               'naissances': (lambda x: int(float(x)))})
df = df[(df['année'] != 'XXXX')&(df['prénom'] != '_prenoms_rares')&(df['prénom'].str.len() != 1)]
df = df [['année', 'prénom', 'genre', 'naissances']]
df.sort_values(by=['année', 'genre', 'naissances', 'prénom'], ascending=[True, True, False, True], inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
# les 10 prénoms les plus donnés dans une année
df.sort_values(by='naissances', ascending=False).head(10)

In [ ]:
# nombre de naissances par genre
df.pivot_table(index='genre', values=['naissances'], aggfunc=np.sum)

In [ ]:
# les 10 prénoms + genre les plus donnés
data = df.pivot_table(index=['prénom','genre'], values=['naissances'], aggfunc=np.sum)
data.sort_values(by='naissances', ascending=False, inplace=True)
data.reset_index(inplace=True)
data.head(10)

In [ ]:
cherche("Jo") # Nath Fran Em

In [ ]:
# naissances par année et par genre
total = df.pivot_table(index='année', columns=['genre'], values='naissances', aggfunc=np.sum)
total

In [ ]:
# graphique
total.plot(title="Total des naissances par année et par genre", color=['r','b']);

In [ ]:
stats_prenom('Sarah', 'F')

## Analyse de tendances

In [ ]:
df = df.groupby(['année', 'genre']).apply(ajout_proportion)
df.head()

In [ ]:
# vérification : la somme des proportions par année et par genre vaut presque 1
np.allclose(df.groupby(['année', 'genre'])["proportion"].sum(), 1)

In [ ]:
df1000 = df.groupby(['année', 'genre']).apply(top1000)
df1000.head()

In [ ]:
# évolution des prénoms par année
data = df1000.pivot_table(index='année', columns='prénom', values='naissances', aggfunc=sum)
data = data[['Jean', 'Pierre', 'Marie', 'Jeanne']]
data.head()

In [ ]:
# graphique
data.plot(subplots=True, figsize=(12, 10), grid=False, title="Nombre de naissances par an");

## Mesure de l'évolution de la diversité des prénoms

In [ ]:
# nombre de prénoms différents par année et genre
data = df.pivot_table(index='année', columns='genre', values='prénom', aggfunc=lambda x: len(x))
data.head()

In [ ]:
data.tail()

In [ ]:
# graphique
data.plot(title="Evolution du nombre de prénoms distincts par genre",
         color=['r', 'b']);

In [ ]:
# proportion des naissances qui sont dans le top 1000 par année et par genre
data = df1000.pivot_table(index='année', columns='genre', values='proportion', aggfunc=sum)
data.head()

In [ ]:
data.tail()

In [ ]:
# graphique
data.plot(title='Proportion des naissances dans le top 1000 par année et genre',
          xticks=range(1900, 2020, 10),
          yticks=np.linspace(0, 1.2, 13),
          figsize=(12, 6),
          color=['r','b']);

In [ ]:
# calcul de la diversité
diversite = df1000.groupby(['année', 'genre']).apply(quantile_compte)
diversite = diversite.unstack('genre')
diversite.head()

In [ ]:
diversite.tail()

In [ ]:
# graphique
diversite.plot(title="Nombre de prénoms populaires dans le top 50%",
          color=['r','b']);

## Evolution de la dernière lettre des prénoms¶

In [ ]:
# ajout d'une colonne avec la lettre terminale
df["terminale"] = df["prénom"].apply(lambda x: x[-1])
df.head()

In [ ]:
# tableau du nombre de prénoms par année x lettre terminale
data = df.pivot_table(index='année', columns='terminale', values='prénom', aggfunc=len)
data.head()

In [ ]:
df.tail()

In [ ]:
# les 7 lettres terminales les plus utilisées en 2015
lettres = data.loc[2015].sort_values(ascending=False).head(7).index.values
lettres

In [ ]:
# graphique
data[lettres].plot(title="Evolution du nombre de prénoms distincts en fonction de leur lettre terminale");

In [ ]:
# tableau letres terminales x (genres, années)
data = df.pivot_table(index='terminale', columns=['genre', 'année'], values='naissances', aggfunc=sum)
data.head()

In [ ]:
# extrait pour les années 1910, 1960 et 2010
subdata = data.reindex(columns=[1910, 1960, 2010], level='année')
subdata

In [ ]:
# proportion du nombre de prénoms par lettre terminale
proportion_lettre = subdata / subdata.sum().astype(float)
proportion_lettre

In [ ]:
# graphique
fig, axes = plt.subplots(2, 1, figsize=(10, 8))
proportion_lettre['M'].plot(kind='bar', rot=0, ax=axes[0], title='Masculin')
proportion_lettre['F'].plot(kind='bar', rot=0, ax=axes[1], title='Féminin', legend=False);

In [ ]:
# proportion du nombre de prénoms par lettre terminale
proportion_lettre = data / data.sum().astype(float)
proportion_lettre

In [ ]:
# proportion pour les lettres n, e et s (M)
nes = proportion_lettre.ix[['n', 'e', 's'], 'M'].T
nes

In [ ]:
# graphique
nes.plot(title="Proportion des hommes avec des prénoms se terminant en n, e ou s");

In [ ]:
# proportion pour les lettres a, e et s (F)
aes = proportion_lettre.ix[['a', 'e', 's'], 'F'].T
aes

In [ ]:
# graphique
aes.plot(title="Proportion des femmes avec des prénoms se terminant en a, e ou s");

## Prénoms dont le genre a évolué

In [ ]:
# ratio par prénom et par an du genre F
data = df1000.pivot_table(index=['prénom','année'], columns=['genre'], values='naissances', aggfunc=np.sum)
data.dropna(inplace=True)
data["ratio"] = data["F"] / (data["F"]+data["M"])
data.reset_index(inplace=True)
data

In [ ]:
# variance du ratio par prénom
data = data.pivot_table(index='prénom', values=['ratio'], aggfunc=np.var)
data.dropna(inplace=True)
data.sort_values(by="ratio", ascending=False, inplace=True)
data.head()

In [ ]:
evolution("Camille")

In [ ]:
evolution("Dominique")

In [ ]:
evolution("Alix")

## Comparaison du nombre de naissances entre 2 années sucessives

In [ ]:
# années 2014 et 2015
var = df[df["année"].isin([2014,2015])].pivot_table(index="prénom",columns="année",values="naissances")
var.fillna(0, inplace=True)
var /=1000
var.head()

In [ ]:
# graphique
var.plot(kind='scatter', x=2014, y=2015);

In [ ]:
# régression linéaire
from scipy.stats import linregress
slope, intercept, r, p, stderr = linregress(var[2014], var[2015])
slope, intercept

In [ ]:
# graphique
plt.scatter(x=var[2014], y=var[2015]);
x = np.linspace(0,5,2)
n = slope * x + intercept
plt.plot(x, n, 'y');

## Comparaison France / USA

In [ ]:
# chargement de toute les données dans un DataFrame
dfusa = pnd.DataFrame()
for root, dirs, files in os.walk(path):
    for filename in files:
        annee = int(filename[3:7]) # yobAAAA.txt
        csv = pnd.read_csv(os.path.join(root, filename),
                           header=None,
                           names=['prénom', 'genre', 'naissances'])
        csv['année'] = annee
        dfusa = dfusa.append(csv, ignore_index=True)
dfusa = dfusa[['année', 'prénom', 'genre', 'naissances']]
# tête du DataFrame
dfusa.head()

In [ ]:
df.head()

In [ ]:
def ajout_proportion(groupe):
    # Integer division floors
    naissances = groupe["naissances"].astype(float)
    groupe['proportion'] = naissances / naissances.sum()
    return groupe

In [ ]:
var = df.groupby(['année', 'genre']).apply(ajout_proportion)
var.set_index(['année','prénom','genre'], drop=True, inplace=True)
var.head()

In [ ]:
varusa = dfusa.groupby(['année', 'genre']).apply(ajout_proportion)
varusa.set_index(['année','prénom','genre'], drop=True, inplace=True)
varusa.head()

In [ ]:
merge = var.join(varusa, lsuffix='_fr', rsuffix='_usa')[['proportion_fr','proportion_usa']]
merge = merge.unstack('année').stack('année')
merge

In [ ]:
import warnings
warnings.simplefilter("error")
corr = pnd.DataFrame()
for name, group in merge.groupby(level=[0, 1]):
    try:
        x = group['proportion_fr'].corr(group['proportion_usa'])
        if pnd.notnull(x):
            row = list(name)
            row.append(x)
            corr = corr.append([row])
    except:
        pass
corr

In [ ]:
corr.columns = ['prénom', 'genre','corrélation']
corr.reset_index(drop=True, inplace=True)
corr.sort_values(by='corrélation', inplace=True)
corr.reset_index(drop=True, inplace=True)
corr

In [ ]:
corr.plot()